<a href="https://colab.research.google.com/github/eekaiboon/gen_ai/blob/main/export_error.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal

Provide suggestion on how to resolve export error.

# Scenario

Try to export the following expense to Quickbooks Online:

```
{
    "time": "2015-06-29T12:43:42.132-07:00",
    "JournalEntry": {
        "SyncToken": "0",
        "domain": "QBO",
        "TxnDate": "2015-06-29",
        "sparse": false,
        "Line": [
            {
                "Description": "Four sprinkler heads damaged",
                "JournalEntryLineDetail": {
                    "PostingType": "Debit",
                    "AccountRef": {
                        "name": "Job Expenses:Job Materials:Fountain and Garden Lighting",
                        "value": "65"
                    },
                    "Entity": {
                        "Type": "Customer",
                        "EntityRef": {
                            "name": "Amy's Bird Sanctuary",
                            "value": "1"
                        }
                    }
                },
                "DetailType": "JournalEntryLineDetail",
                "ProjectRef": {
                    "value": "39298034"
                },
                "Amount": 25.54,
                "Id": "0"
            },
            {
                "JournalEntryLineDetail": {
                    "PostingType": "Credit",
                    "AccountRef": {
                        "name": "Notes Payable",
                        "value": "44"
                    }
                },
                "DetailType": "JournalEntryLineDetail",
                "Amount": 25.54,
                "Id": "1",
                "Description": "Sprinkler Hds - Sprinkler Hds Inventory Adjustment"
            }
        ],
        "Adjustment": false,
        "Id": "227",
        "TxnTaxDetail": {},
        "MetaData": {
            "CreateTime": "2015-06-29T12:33:57-07:00",
            "LastUpdatedTime": "2015-06-29T12:33:57-07:00"
        }
    }
}
```

But export fails with the following error:

```
Business Validation Error: When you use Accounts Payable, you must choose a vendor in the Name field.
```



# Query engine approach: LLM response based on provided API documentation

*   In this example, we have 2 API documentation: (1) QBO API documentation (2) Netsuite API documentation. These 2 API documentations are stored in our "knowledge base". We will let LLM to decide which documentation to reference when resolving export error of specific ERP (e.g. when resolving QBO export error, it should have reference the QBO API documentation).
*   We probably don't need to provide API documentation in prompt if the LLM is API documentation is publicly available on the internet (i.e. not behind paywall) and the API version that we use is covered by the LLM knowledge cutoff date (e.g. the knowledge cutoff date for OpenAI GPT-4 Turbo is April 2023).

In [16]:
!pip install --upgrade llama-index

Defaulting to user installation because normal site-packages is not writeable


In [11]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [28]:
from llama_index import load_index_from_storage

try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/export_error/qbo_api"
    )
    qbo_api_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/export_error/netsuite"
    )
    netsuite_api_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

print(index_loaded)

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
True


In [19]:
!mkdir -p 'data/export_error/'
!wget 'https://drive.google.com/file/d/1Z6Gyv6rghFTrVECEEqOBfGOkWNtCLTE-/view?usp=sharing' -O 'data/export_error/qbo_journal_entry.md'
!wget 'https://drive.google.com/file/d/1FYfbclnIooUN7mqlZErURHlovKyz7d9p/view?usp=sharing' -O 'data/export_error/netsuite_journal_entry.md'

--2023-11-08 15:05:48--  https://drive.google.com/file/d/1Z6Gyv6rghFTrVECEEqOBfGOkWNtCLTE-/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 2607:f8b0:4005:814::200e, 142.250.189.174
Connecting to drive.google.com (drive.google.com)|2607:f8b0:4005:814::200e|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘data/export_error/qbo_journal_entry.md’

data/export_error/q     [ <=>                ]  82.44K  --.-KB/s    in 0.03s   

2023-11-08 15:05:49 (2.91 MB/s) - ‘data/export_error/qbo_journal_entry.md’ saved [84415]

--2023-11-08 15:05:49--  https://drive.google.com/file/d/1FYfbclnIooUN7mqlZErURHlovKyz7d9p/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 2607:f8b0:4005:814::200e, 142.250.189.174
Connecting to drive.google.com (drive.google.com)|2607:f8b0:4005:814::200e|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘data/export_error/ne

In [29]:
from llama_index import SimpleDirectoryReader, VectorStoreIndex, StorageContext

if not index_loaded:
    # load data
    qbo_api_docs = SimpleDirectoryReader(
        input_files=["./data/export_error/qbo_journal_entry.md"]
    ).load_data()
    netsuite_api_docs = SimpleDirectoryReader(
        input_files=["./data/export_error/netsuite_journal_entry.md"]
    ).load_data()

    # build index
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/export_error/qbo_api"
    )
    qbo_api_index = VectorStoreIndex.from_documents(qbo_api_docs, storage_context=storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/export_error/netsuite"
    )
    netsuite_api_index = VectorStoreIndex.from_documents(netsuite_api_docs, storage_context=storage_context)

In [30]:
journal_entry_payload = """
{
    "time": "2015-06-29T12:43:42.132-07:00",
    "JournalEntry": {
        "SyncToken": "0",
        "domain": "QBO",
        "TxnDate": "2015-06-29",
        "sparse": false,
        "Line": [
            {
                "Description": "Four sprinkler heads damaged",
                "JournalEntryLineDetail": {
                    "PostingType": "Debit",
                    "AccountRef": {
                        "name": "Job Expenses:Job Materials:Fountain and Garden Lighting",
                        "value": "65"
                    },
                    "Entity": {
                        "Type": "Customer",
                        "EntityRef": {
                            "name": "Amy's Bird Sanctuary",
                            "value": "1"
                        }
                    }
                },
                "DetailType": "JournalEntryLineDetail",
                "ProjectRef": {
                    "value": "39298034"
                },
                "Amount": 25.54,
                "Id": "0"
            },
            {
                "JournalEntryLineDetail": {
                    "PostingType": "Credit",
                    "AccountRef": {
                        "name": "Notes Payable",
                        "value": "44"
                    }
                },
                "DetailType": "JournalEntryLineDetail",
                "Amount": 25.54,
                "Id": "1",
                "Description": "Sprinkler Hds - Sprinkler Hds Inventory Adjustment"
            }
        ],
        "Adjustment": false,
        "Id": "227",
        "TxnTaxDetail": {},
        "MetaData": {
            "CreateTime": "2015-06-29T12:33:57-07:00",
            "LastUpdatedTime": "2015-06-29T12:33:57-07:00"
        }
    }
}
"""

export_error = "Business Validation Error: When you use Accounts Payable, you must choose a vendor in the Name field."

system_prompt = """
You are expert in Quickbooks API integration.
Please respond to question related to Quickbooks API integration based on the official Quickbooks API documentation.
"""

user_prompt = f"""
We are trying to create the following journal entry object in Quickbooks Online using API call.

Journal entry object request payload:
```json
{journal_entry_payload}
```

However, we ran into the following error when create the journal entry object via API.

API error response:
{export_error}

How should I fix the error?

Please do 2 things:
1. Please provide an updated request payload.
2. Please provide a user friendly error that I can use to display in my web application.
""".replace(
    "{", "{{"
).replace(
    "}", "}}"
)

print(user_prompt)


We are trying to create the following journal entry object in Quickbooks Online using API call.

Journal entry object request payload:
```json

{{
    "time": "2015-06-29T12:43:42.132-07:00",
    "JournalEntry": {{
        "SyncToken": "0",
        "domain": "QBO",
        "TxnDate": "2015-06-29",
        "sparse": false,
        "Line": [
            {{
                "Description": "Four sprinkler heads damaged",
                "JournalEntryLineDetail": {{
                    "PostingType": "Debit",
                    "AccountRef": {{
                        "name": "Job Expenses:Job Materials:Fountain and Garden Lighting",
                        "value": "65"
                    }},
                    "Entity": {{
                        "Type": "Customer",
                        "EntityRef": {{
                            "name": "Amy's Bird Sanctuary",
                            "value": "1"
                        }}
                    }}
                }},
            

In [14]:
from llama_index import LLMPredictor, ServiceContext
from llama_index.llms import OpenAI
from llama_index.tools import QueryEngineTool
from llama_index.query_engine import RouterQueryEngine

from llama_index.selectors.llm_selectors import LLMSingleSelector

llm = OpenAI(model="gpt-3.5-turbo-1106")
llm_predictor = LLMPredictor(system_prompt=system_prompt, llm=llm)

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

# Define tools.
qbo_tool = QueryEngineTool.from_defaults(
    query_engine=qbo_api_index.as_query_engine(service_context=service_context),
    description="Use this query engine to answer Quickbooks Online related API questions",
)
netsuite_tool = QueryEngineTool.from_defaults(
    query_engine=netsuite_api_index.as_query_engine(service_context=service_context),
    description="Use this query engine to answer Netsuite related API questions",
)

# Define query engine.
# Define a RouterQueryEngine over the tools.
# By default, this uses a LLMSingleSelector as the router, which uses the LLM to choose the best sub-index to router the query to, given the descriptions.
query_engine = RouterQueryEngine.from_defaults(
    selector=LLMSingleSelector.from_defaults(service_context=service_context),
    query_engine_tools=[qbo_tool, netsuite_tool]
)

response = query_engine.query(user_prompt)
print(response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.query_engine.router_query_engine:Selecting query engine 0: Use this query engine to answer Quickbooks Online related API questions.
Selecting query engine 0: Use this query engine to answer Quickbooks Online related API questions.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
1. Updated request payload:
```json
{
    "time": "2015-06-29T12:43:42.132-07:00",
    "JournalEntry": {
        "SyncToken": "0",
        "domain": "QBO",
        "TxnDate": "2015-06-29",
        "sparse": false,
        "Line": [
            {
     

# Reference


1.   [Simple Website Loader](https://llamahub.ai/l/web-simple_web)
2.   [LlamaIndex OpenAI Colab](https://github.com/run-llama/llama_index/blob/main/docs/examples/llm/openai.ipynb)
3.   [Install wget on M1](https://stackoverflow.com/questions/64963370/error-cannot-install-in-homebrew-on-arm-processor-in-intel-default-prefix-usr)
4.   [OpenAI Agent with Query Engine Tools](https://github.com/run-llama/llama_index/blob/2d30e96671d2b53a5299801969f39a7d336cf9f8/docs/examples/agent/openai_agent_with_query_engine.ipynb)
5.   [QBO JournalEntry API doc](https://developer.intuit.com/app/developer/qbo/docs/api/accounting/all-entities/journalentry)
6.   [LlamaIndex Q&A patterns](https://docs.llamaindex.ai/en/stable/understanding/putting_it_all_together/q_and_a.html#routing-over-heterogeneous-data)